In [ ]:
from nucli_train.data_management.builders import build_data
from nucli_train.models.builders import build_model, MODEL_REGISTRY, MODEL_BUILDERS_REGISTRY

from nucli_train.training import Trainer

import voco_transform as voto


import numpy as np
import yaml

a
a
a
a
a
a
a
a
a
a


/Users/adammesbahi/miniforge3/envs/nnssl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a


In [2]:
from nucli_train.preprocess.preprocessor import PreprocessorBlosc2
import nucli_train.val.evaluators

In [ ]:
class VoCoPreprocessor(PreprocessorBlosc2): 
    def __init__(self, **kwargs): 
        super().__init__(**kwargs)
    
    def exclude_condition(self, nifti_filename):
        return nifti_filename.endswith("0000.nii.gz") 
    
    def identify_tracer(self, nifti_filename):
        if "fdg" in nifti_filename:
            return "fdg"
        elif "psma" in nifti_filename:
            return "psma"
        return "unknown"
        
    
    def identify_center(self, nifti_filename):
        return "autopet_center"
    


kwargs = {
    "dataset_name": "autopet_2024",
    "dataset_path": "/home/interns_nuclivision_com/aws_backup/adam/autopet_2024",
    "exp_name": "VOCO_adam_experiment",
    "nifti_input_rootdir": "/home/interns_nuclivision_com/aws_backup/adam/autopet_2024/imagesTr",
    "nifti_target_rootdir": None,
    "percentage_dataset": 1.0,
    "train_val_percentage": 0.8,
    "spacing": (1.0, 1.0, 1.0),
    "batch_size_train": 8,
    "batch_size_val": 8,
    "num_workers_train": 2,
    "num_workers_val": 2,
    "global_eval_interval": 1,
    "patch_size": (64, 64, 64),
    "shuffle_pick": True,
    "validation_evaluator": "save-preds-VOCO", 
    "transformation": "voco_transform", 
    "transformation_args": {
        "voco_base_crop_count": (2, 2, 2), 
        "voco_crop_size": (32, 32, 32),
        "voco_target_crop_count": 4
    }
}


In [4]:
my_preprocessor = VoCoPreprocessor(**kwargs)

VOCO_adam_experiment
Compressed fdg_0cda25453b_10-12-2003-NA-PET-CT Ganzkoerper  primaer mit KM-47333_0001.nii.gz to /Users/adammesbahi/Desktop/nucli-ssl/data/autopet_2024/nucli_train/VOCO_adam_experiment/autopet_2024/blosc2_128_128_32/training/autopet_2024/autopet_center/fdg/input/fdg_0cda25453b_10-12-2003-NA-PET-CT Ganzkoerper  primaer mit KM-47333_0001
Shape of the volume: (400, 400, 326)
Adam -- -- -- Number of valid centers: 661427
saved coordinates to /Users/adammesbahi/Desktop/nucli-ssl/data/autopet_2024/nucli_train/VOCO_adam_experiment/autopet_2024/blosc2_128_128_32/training/autopet_2024/autopet_center/fdg/coords/fdg_0cda25453b_10-12-2003-NA-PET-CT Ganzkoerper  primaer mit KM-47333_0001.npy

 
 
 
Compressed fdg_08cdb15e0b_11-21-2003-NA-PET-CT Ganzkoerper  primaer mit KM-47234_0001.nii.gz to /Users/adammesbahi/Desktop/nucli-ssl/data/autopet_2024/nucli_train/VOCO_adam_experiment/autopet_2024/blosc2_128_128_32/training/autopet_2024/autopet_center/fdg/input/fdg_08cdb15e0b_11-21-20

In [5]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
from nucli_train.models.image_translation import ImageTranslationModel

import torch

from einops import rearrange


class MIM(ImageTranslationModel):
        
    def train_step(self, batch):
        data = batch['all_crops'].to(device)
        base_crop_index = batch['base_crop_index'].to(device)
        overlaps = batch["base_target_crop_overlaps"].to(device)

        base_list = []
        target_list = []
        
        for i in range(data.shape[0]):
            base_crop = data[i, :base_crop_index[i], :, :, :]
            target_crops = data[i, base_crop_index[i]:, :, :, :]
            base_list.append(base_crop)
            target_list.append(target_crops)
        
        batch_size = data.shape[0]
        NBASE = base_crop_index[0].item()  # Number of base crops
        nTARGET = data.shape[1] - NBASE  # Number of target crops
        
        
        
        # Concaténer tous les éléments le long de la première dimension
        base_tensor = torch.cat(base_list, dim=0)    
        target_tensor = torch.cat(target_list, dim=0) 


        base_embeddings = self.network(base_tensor[:, None, :, :, :])  # Assuming the network expects a 5D tensor
        target_embeddings = self.network(target_tensor[:, None, :, :, :])

        base_embeddings = rearrange(base_embeddings, "(b NBASE) c 1 1 1 -> b NBASE c", b=batch_size)
        target_embeddings = rearrange(target_embeddings, "(b nTARGET) c 1 1 1 -> b nTARGET c", b=batch_size)

        print("overlaps size:", overlaps.shape)

        losses = self.get_losses_several_inputs({"base_embeddings": base_embeddings, "target_embeddings": target_embeddings, "gt_overlaps": overlaps})

        return losses

    def validation_step(self, batch):
        data = batch['all_crops'].to(device)
        base_crop_index = batch['base_crop_index'].to(device)
        overlaps = batch["base_target_crop_overlaps"].to(device)

        base_list = []
        target_list = []
        
        for i in range(data.shape[0]):
            base_crop = data[i, :base_crop_index[i], :, :, :]
            target_crops = data[i, base_crop_index[i]:, :, :, :]
            base_list.append(base_crop)
            target_list.append(target_crops)
        
        batch_size = data.shape[0]
        NBASE = base_crop_index[0].item()  # Number of base crops
        nTARGET = data.shape[1] - NBASE  # Number of target crops

        num_base = base_crop_index[0].item()
        num_target = data.shape[1] - num_base
        
        
        
        # Concaténer tous les éléments le long de la première dimension
        base_tensor = torch.cat(base_list, dim=0)    
        target_tensor = torch.cat(target_list, dim=0) 


        base_embeddings = self.network(base_tensor[:, None, :, :, :])  # Assuming the network expects a 5D tensor
        target_embeddings = self.network(target_tensor[:, None, :, :, :])
        base_embeddings = rearrange(base_embeddings, "(b NBASE) c 1 1 1 -> b NBASE c", b=batch_size)
        target_embeddings = rearrange(target_embeddings, "(b nTARGET) c 1 1 1 -> b nTARGET c", b=batch_size)

        all_losses = self.get_losses_several_inputs({"base_embeddings": base_embeddings, "target_embeddings": target_embeddings, "gt_overlaps": overlaps})

        return {
            "losses": all_losses,
            "pred_loss": all_losses["info"]["pred_loss"],
            "reg_loss": all_losses["info"]["reg_loss"],
            "base_embeddings": base_embeddings,
            "target_embeddings": target_embeddings,
            "overlaps": overlaps, 
            "bases": base_tensor[:num_base],
            "targets": target_tensor[:num_target],
            "num_base": num_base,
            "num_target": num_target
        }

In [7]:
from nucli_train.nets import build_network
from nucli_train.models.losses import build_losses

@MODEL_BUILDERS_REGISTRY.register('VoCo')
def build_VoCo(cfg):
    network = build_network(cfg['args']['network'])

    losses = build_losses(cfg['args']['losses'])    

    return MIM(network, loss_functions=losses)

a


In [ ]:
model = build_model('/home/interns_nuclivision_com/nucli-ssl/examples/VOCO/MIM_model.yaml')

{'image_translation': <function build_image_translation_model at 0x160247d00>, 'voco': <function build_VoCo at 0x1605ad480>}
VoCo
{'unet': <function build_unet_from_cfg at 0x160219c60>, 'encoder_unet': <function build_encoder_unet_from_cfg at 0x160219fc0>}
encoder_unet
{'perceptuallossmedicalnet': <class 'nucli_train.models.losses.perceptual.PerceptualLoss3D'>, 'l1loss': <function l1_loss at 0x160219240>, 'vocoloss': <function voco_loss at 0x1602192d0>}
vocoloss
VoCo loss initialized


In [ ]:
from nucli_train.data_management.builders import build_data
import nucli_train.data_management.transformations 

train_data, val_loaders = build_data("/home/interns_nuclivision_com/aws_backup/adam/autopet_2024/nucli_train/VOCO_adam_experiment/autopet_2024" + "/main.yaml")

a
SavePredictionVOCO initialized


In [10]:
import mlflow

In [ ]:
mlflow.end_run()

trainer = Trainer(model, train_data=train_data, val_loaders=val_loaders, run_name='base-deeper', experiment_name='VoCo', save_interval=1, model_cfg_path='/Users/adammesbahi/Desktop/nucli-ssl/nucli-ssl/examples/VOCO/MIM_model.yaml', data_cfg_path="/home/interns_nuclivision_com/aws_backup/adam/autopet_2024/nucli_train/VOCO_adam_experiment/autopet_2024" + "/main.yaml")

Traceback (most recent call last):
  File "/Users/adammesbahi/miniforge3/envs/nnssl/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 356, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/Users/adammesbahi/miniforge3/envs/nnssl/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 454, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/Users/adammesbahi/miniforge3/envs/nnssl/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1595, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/Users/adammesbahi/miniforge3/envs/nnssl/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1588, in _read_helper
    result = read_yaml(root, file_name)
  File "/Users/adammesbahi/miniforge3/envs/nnssl/lib/python3.10/site-packages/mlflow/utils/yaml_utils.py", line 107, in read_yaml
    raise MissingCon

In [12]:
trainer.run(1000)

Epoch 1:   0%|          | 0/16 [00:00<?, ?batch/s]

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
aa

a
a
a
a
a
overlaps size: torch.Size([2, 1, 4, 16])
Logits shape: torch.Size([2, 4, 16])


Epoch 1:   6%|▋         | 1/16 [00:10<02:37, 10.51s/batch]

overlaps size: torch.Size([2, 1, 4, 16])
Logits shape: torch.Size([2, 4, 16])


Epoch 1:  12%|█▎        | 2/16 [00:16<01:53,  8.12s/batch]

...

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
aa

a
aa

overlaps size: torch.Size([2, 1, 4, 16])
Logits shape: torch.Size([2, 4, 16])


Epoch 59:   6%|▋         | 1/16 [00:11<02:45, 11.01s/batch]

overlaps size: torch.Size([2, 1, 4, 16])
Logits shape: torch.Size([2, 4, 16])


Epoch 59:   6%|▋         | 1/16 [00:28<07:04, 28.30s/batch]


OSError: [Errno 28] No space left on device